##### produce driver stanadings

In [0]:
from pyspark.sql.functions import sum, when, col, count, desc, rank
from pyspark.window import Window

In [0]:
race_results_df = spark.read.parquet(f"{gold_container_path}")

In [0]:
drivers_standing_df = race_results_df\
                      .groupBy("race_year", "driver_name", "driver_nationality", "team")
                      .agg(sum("points").alias("total_points"), count(when(col("position") == 1, True)).alias("wins"))

In [0]:
driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = drivers_standing_df.withColumn("rank", rank().over(driver_rank_spec))

In [0]:
final_df.write.mode("overwrite").parquet(f"{gold_container_path}")